In [3]:
import numpy as np
import cv2
import mediapipe as mp
import time
import math

In [1]:
import comtypes
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume

comtypes.CoInitialize()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

volume = cast(interface, POINTER(IAudioEndpointVolume))

    # Now you can use volume control methods here
volume_range = volume.GetVolumeRange()
print(f"Volume Range: {volume_range}")
min_vol = volume_range[0]
max_vol = volume_range[1]

comtypes.CoUninitialize()

Volume Range: (-63.5, 0.0, 0.5)


In [4]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

Cam_width,Cam_height= 1200,1000

cap = cv2.VideoCapture(0)
cap.set(3,Cam_width)
cap.set(4,Cam_height)

previous_Time=0
vol=0
vol_percentage=0
vol_bar=400
ret,frame = cap.read()
while ret:

    current_time = time.time()
    fps = 1/(current_time-previous_Time)
    previous_Time=current_time

    cv2.putText(frame,f'FPS : {int(fps)}',(40,70),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(255,0,0),3)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
        h, w, c = frame.shape  # get frame dimensions

        # Thumb (landmark 4) and index finger (landmark 8) landmarks
        x1, y1 = int(hand_landmarks.landmark[4].x * w), int(hand_landmarks.landmark[4].y * h)
        x2, y2 = int(hand_landmarks.landmark[8].x * w), int(hand_landmarks.landmark[8].y * h)


        cv2.circle(frame,(x1,y1),15,(255,0,255),cv2.FILLED)
        cv2.circle(frame,(x2,y2),15,(255,0,255),cv2.FILLED)
        cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),3)

        cx,cy= (x1+x2)//2,(y1+y2)//2     # Centre of line to put another circle in centre
        cv2.circle(frame,(cx,cy),15,(255,0,255),cv2.FILLED)

        # Now based on the length of line we can change our volume
        length = math.hypot(x2-x1,y2-y1)
        # print(length)       # our max length is around 180 and min length is around 40

        vol = np.interp(length,[40,180],[min_vol,max_vol])
        vol_bar = np.interp(length,[40,180],[400,150])
        vol_percentage = np.interp(length,[40,180],[0,100])
        # print(int(length),vol)
        volume.SetMasterVolumeLevel(vol,None)

        if length<40:
            cv2.circle(frame,(cx,cy),15,(0,255,0),cv2.FILLED)

    cv2.rectangle(frame,(40,150),(85,400),(0,255,0),3)
    cv2.rectangle(frame,(40,int(vol_bar)),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(frame,f'Vol : {int(vol_percentage)} %',(40,450),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(255,0,0),3)

    cv2.imshow('Frame',frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):      
        break

    ret,frame = cap.read()

cap.release()
cv2.destroyAllWindows()
        


c:\Users\Shahzaib\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
